In [8]:
## Lets import our requirements
import boto3
import pandas as pd
#import psycopg2
import pymysql
from sqlalchemy import create_engine
import os
import sqlite3
import sys
parentDirectory = os.path.abspath(os.path.join(os.getcwd(), os.pardir))
sys.path.insert(0,parentDirectory)
from secretsPaul import secrets


import re
import gzip
import shutil
import urllib


In [2]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)

In [3]:
#connections
def create_connection:
    from secretsPaul import secrets
    endpoint = secrets.get('DATABASE_ENDPOINT')
    user = secrets.get('DATABASE_USER')
    password = secrets.get('DATABASE_PASSWORD')
    port=secrets.get('DATABASE_PORT')
    connection = create_engine(f'mysql+pymysql://{user}:{password}@{endpoint}:{port}/Capstone', pool_recycle=3600);
    return connection

In [20]:
def handle_listing(listing_detail):
    listing_detail.rename(columns={'id':'listing_id', 'name':'listing_name'})
    listing_detail['price']=listing_detail['price'].str[1:].str.replace(',','').apply(pd.to_numeric) #changing price column to numeric field
    listing_detail['host_acceptance_rate']=listing_detail['host_acceptance_rate'].str[:-1].apply(pd.to_numeric)/100 #change to numeric
    listing_detail['host_response_rate']=listing_detail['host_response_rate'].str[:-1].apply(pd.to_numeric)/100

    #converting to datetime
    listing_detail[['last_scraped', 'host_since','calendar_last_scraped','first_review','last_review']]=listing_detail[['last_scraped', 'host_since','calendar_last_scraped','first_review','last_review']].apply(pd.to_datetime)
    
    #converting to 1/0 (better for modeling and summarization)
    listing_detail[['host_is_superhost','host_has_profile_pic','host_identity_verified','has_availability','instant_bookable']]=listing_detail[['host_is_superhost','host_has_profile_pic','host_identity_verified','has_availability','instant_bookable']].replace('t',1).replace('f',0)

    listing_detail.to_sql('listing_detail_stage',con=connection, if_exists='replace',index=False)


In [21]:
def handle_calendar(calendar):
    calendar['price']=calendar['price'].str[1:].str.replace(',','').apply(pd.to_numeric) #changing price column to numeric field
    calendar['adjusted_price']=calendar['adjusted_price'].str[1:].str.replace(',','').apply(pd.to_numeric) #changing price column to numeric field
    calendar['date']=calendar['date'].apply(pd.to_datetime)
    calendar['available']=calendar['available'].replace('t',1).replace('f',0)
    calendar.rename(columns={'date':'calendar_date'})

    calendar.to_sql('calendar_stage',con=connection, if_exists='replace',index=False)

In [22]:
def handle_reviews(reviews):
    reviews['date']=reviews['date'].apply(pd.to_datetime)
    reviews.rename(columns={'date':'review_date'})
    
    reviews.to_sql('reviews_stage',con=connection, if_exists='replace',index=False)

In [23]:
filelist =  ['http://data.insideairbnb.com/united-states/tx/austin/2022-12-15/data/listings.csv.gz',
 'http://data.insideairbnb.com/united-states/tx/austin/2022-12-15/data/calendar.csv.gz',
  'http://data.insideairbnb.com/united-states/tx/austin/2022-12-15/data/reviews.csv.gz']



In [24]:
for file in filelist:
    file_split = file[46:].split("/")
    file_name = file_split[0]+'_'+file_split[1]+'_'+file_split[3]


    urllib.request.urlretrieve(file, file_name)

    with gzip.open(file_name, 'rb') as f_in:
        with open(file_name[:-3], 'wb') as f_out:
            shutil.copyfileobj(f_in, f_out)
            print(file_name)
            df = pd.read_csv(file_name)
            if "listing" in file_name:
                handle_listing(df)
            if "calendar" in file_name:
                handle_calendar(df)
            if "review" in file_name:
                handle_reviews(df)


austin_2022-12-15_listings.csv.gz
austin_2022-12-15_calendar.csv.gz


Exception during reset or similar
Traceback (most recent call last):
  File "c:\Users\Patrick.Murphy\AppData\Local\Programs\Python\Python38\lib\site-packages\pymysql\connections.py", line 732, in _read_bytes
    data = self._rfile.read(num_bytes)
  File "c:\Users\Patrick.Murphy\AppData\Local\Programs\Python\Python38\lib\socket.py", line 669, in readinto
    return self._sock.recv_into(b)
ConnectionResetError: [WinError 10054] An existing connection was forcibly closed by the remote host

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "c:\Users\Patrick.Murphy\AppData\Local\Programs\Python\Python38\lib\site-packages\sqlalchemy\pool\base.py", line 753, in _finalize_fairy
    fairy._reset(pool)
  File "c:\Users\Patrick.Murphy\AppData\Local\Programs\Python\Python38\lib\site-packages\sqlalchemy\pool\base.py", line 1004, in _reset
    pool._dialect.do_rollback(self)
  File "c:\Users\Patrick.Murphy\AppData\Local\Programs\Python\Py

austin_2022-12-15_reviews.csv.gz
